In [ ]:
import pandas as pd
import matplotlib

### Get shares

In [ ]:
reshare = pd.read_csv('../../data/owid/share-of-electricity-production-from-renewable-sources.csv')

In [ ]:
reshare.head()

In [ ]:
reshare = reshare[reshare.Year == 2022]
reshare = reshare.dropna(subset=['Code'])
reshare = reshare.rename(columns={'Renewables - % electricity': 'reshare'})

### Get total demand

In [ ]:
edemand = pd.read_csv('../../data/owid/electricity-demand.csv')

In [ ]:
edemand.head()

In [ ]:
edemand = edemand[edemand.Year == 2022]
# Drop rows where Code is NaN
edemand = edemand.dropna(subset=['Code'])
edemand = edemand.rename(columns={'Electricity demand - TWh': 'edemand'})

### Merge electricity demand and shares on "Code"

In [ ]:
merge = pd.merge(reshare, edemand, on='Code')

In [ ]:
merge.head()

In [ ]:
merge = merge[["Code", "reshare", "edemand"]]

In [ ]:
# Drop row where code is "OWID_WRL"
#merge = merge[merge.Code != "OWID_WRL"]

In [ ]:
import matplotlib.pyplot as plt

# Extracting countries, x values, and y values
countries = list(merge.Code)
x_values = merge.edemand
y_values = merge.reshare

# Creating the plot
plt.figure(figsize=(10, 6))
plt.scatter(x_values, y_values)

# Adding country labels to the points
for i, country in enumerate(countries):
    plt.text(x_values[i], y_values[i], country)

# Add a vertical line at 90% and label it "90% threshold"
plt.axhline(y=90, color='r', linestyle='--')
plt.text(5e3, 91, '90% threshold', color='red')

# Labeling the axes
plt.xlabel('Electricity demand in TWh')
plt.ylabel('Share of RE in electricity production (%)')

# Adding a title
# plt.title('Countries X-Y Plot')

plt.xscale('log')

# Showing the plot
plt.grid(True)
plt.show()


### Determine amount of electricity for export under certain rule

In [ ]:
# export
rule = 90
ely_eff = 0.62
re_share = merge.reshare
gen_total = merge.edemand

def get_export(rule):
    export_el = ((re_share-rule)/100 * gen_total)
    export = export_el * ely_eff

    merge['export'] = export
    merge["export"] = merge["export"].clip(lower=0)

    return merge

merge = get_export(rule)

In [ ]:
# Sort by export
merge = merge.sort_values(by='export', ascending=False)

In [ ]:
merge.export.sum()

In [ ]:
# Get the sum of export depending on the rule

export_sum = []
rule_range = range(50, 100, 1)
for rule in rule_range:
    merge = get_export(rule)
    export_sum.append(merge.export.sum())

In [ ]:
export_sum_df = pd.DataFrame({'rule': rule_range, 'export_sum': export_sum})
export_sum_df.plot(x='rule', y='export_sum', kind='line')

In [ ]:
merge.head(20)